# Drive Mount

In [1]:
from google.colab import drive
drive.mount('/content/drive')

default_path = '/content/drive/My Drive'

Mounted at /content/drive


# pyLDAvis 설치

- LDA 시각화 패키지

In [ ]:
!pip install -U pyLDAvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 30.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.0.3 which is incompatible.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.25.1 which is incompatible.
tensorflow 2.12.0 

# konlpy 설치

In [ ]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 66.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 43.4 MB/s eta 0:00:00


# Load Data

In [2]:
import pandas as pd
import numpy as np
import re

In [3]:
from pathlib import Path

In [4]:
path1 = Path("/content/drive/MyDrive/메타버스 아카데미/본과정/Python/6. Deep_NLP/miniproj_gpt2/AllData.csv")

data1 = pd.read_csv(str(path1))
data1

,req,res
0,너 좋아하는 차 종류 있어?,무슨 차? 자동차? 마시는 차?
1,ㅋㅋ 마시는 차 말한 거야!,"아하 나 둥글레, 옥수수, 보리차 좋아해"
2,완전 곡물류 좋아하네 ㅋㅋ,야쓰 끓이기 귀찮아서 냉침해 먹어
3,그럼 오래 걸리지 않아?,끓이는 것보다는 훨씬 오래 걸리지 ㅠ
4,근데 냉침 하는 것도 귀찮겠다 ㅜㅠ,응! 그래서 매일은 안 먹고 가끔 마셔
...,...,...
88969,아르바이트 면접 언제야?,내일 아르바이트 면접이야!
88970,무슨 아르바이트 지원했어?,나 카페 아르바이트 지원했어
88971,카페는 경력자 원하지 않아?,맞아 나 전에 카페 경력 일년 정도 있어서 괜찮을걸?
88972,아 맞네 ㅋㅋ 너 카페 일했었네,응 근데 까먹었긴 한데 일 배우면 또 다해


In [ ]:
data1.rename(columns={"req":"Q", "res":"A"}, inplace=True)
data1["label"] = 0

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
path2 = Path("/content/drive/MyDrive/메타버스 아카데미/본과정/Python/6. Deep_NLP/miniproj_gpt2/ChatbotData.csv")

data2 = pd.read_csv(str(path2))
data2

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


In [ ]:
train_data = pd.concat([data2, data1], axis=0)
train_data.reset_index(drop=True, inplace=True)
train_data

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
100792,아르바이트 면접 언제야?,내일 아르바이트 면접이야!,0
100793,무슨 아르바이트 지원했어?,나 카페 아르바이트 지원했어,0
100794,카페는 경력자 원하지 않아?,맞아 나 전에 카페 경력 일년 정도 있어서 괜찮을걸?,0
100795,아 맞네 ㅋㅋ 너 카페 일했었네,응 근데 까먹었긴 한데 일 배우면 또 다해,0


# Preprocessing

In [ ]:
train_data["Q"] = train_data["Q"].str.split("/").str[0]
train_data["A"] = train_data["A"].str.split("/").str[0]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
train_data.shape

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(100797, 3)

In [ ]:
df = train_data

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# 형태소 나누기 (형용사, 동사, 명사)

In [ ]:
import konlpy
import re
from tqdm import tqdm

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
def tokenize_text(text):
    # 우리의 자랑스러운 한글 추출
    text = re.sub(r"[^ㄱ-ㅣ가-힣\s]", "", text)

    okt = konlpy.tag.Okt()
    okt_morphs = okt.pos(text)

    words = list()
    for word, pos in okt_morphs:
        if pos in ["Adjective", "Verb", "Noun"]:
            words.append(word)

    sentence = " ".join(words)
    return sentence

tokenize_text("배달의민족 주문시 리뷰를 자주 참고하는 편입니다.")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'배달 민족 주문 시 리뷰 자주 참고 하는 편입 니'

In [ ]:
token_list = list()
for text in tqdm(df["Q"]):
    token_list.append(tokenize_text(text))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
100%|██████████| 100797/100797 [03:14<00:00, 517.40it/s]


In [ ]:
len(token_list)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


100797

In [ ]:
token_list[1]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'지망 학교 떨어졌어'

##짧은 문장 제외

In [ ]:
boolean_indexing = list()
corpus_list = list()
for idx in range(len(token_list)):
    corpus = token_list[idx]
    boolean_indexing.append(True)

    if len(set(corpus.split())) < 3:
        corpus_list.append(corpus)
        boolean_indexing[idx] = False

for corpus in corpus_list:
    token_list.remove(corpus)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
df[boolean_indexing].shape

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(92902, 3)

In [ ]:
len(token_list)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


92902

## LDA는  count기반의 vetorizer만 사용한다

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# max_df 등장비율, min_df 등장하는 문장 수
count_vectorizer = CountVectorizer(max_df=0.1, min_df=2, max_features=1000, ngram_range=(1, 2))

feat_vec = count_vectorizer.fit_transform(token_list)
feat_vec.shape
# (data 수, column 수)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(92902, 1000)

In [ ]:
# 비슷한 4개의 주제로 그룹화
lda = LatentDirichletAllocation(n_components=4)
lda.fit(feat_vec)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


LatentDirichletAllocation(n_components=4)

In [ ]:
feature_names = count_vectorizer.get_feature_names_out()
feature_names

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array(['가게', '가격', '가고', '가고 싶다', '가고 싶어', '가기', '가끔', '가는', '가는데', '가능',
       '가도', '가려고', '가면', '가보고', '가본', '가봐', '가봤어', '가서', '가슴', '가야',
       '가을', '가자', '가장', '가족', '가지', '간다', '간식', '갈까', '갈래', '감기', '감동',
       '감정', '갑자기', '갔는데', '갔다', '갔어', '강아지', '강원도', '강의', '같기도', '같네',
       '같다', '같더라', '같더라고', '같던데', '같아', '같아 하하', '같아서', '같은', '같은거',
       '같은데', '같지', '개월', '개인', '거기', '거나', '거든', '거래', '거리', '거의', '거임',
       '거지', '걱정', '건가', '건강', '건지', '걸까', '걸려', '검색', '게임', '겨울', '결정',
       '결제', '결혼', '결혼식', '경기', '경우', '경험', '계속', '계절', '계획', '고기',
       '고등학교', '고마워', '고민', '고백', '고생', '고양이', '골프', '공간', '공부', '과자',
       '관련', '관리', '관심', '광고', '괜찮아', '괜찮은', '교육', '교정', '교통', '구경', '구매',
       '군대', '군인', '귀여워', '그거', '그것', '그게', '그냥', '그녀', '그때', '그래',
       '그래 그럼', '그랬어', '그러게', '그런', '그런 가봐', '그런 같아', '그런거', '그런지', '그럴',
       '그럼', '그렇고', '그렇구나', '그렇긴', '그렇지', '그치', '그치 나도', '근처', '금방', '기간',
       '기념일', '기대', '기도', '기본', '기분', '기사', '기억', '기억나', '기차', 

In [ ]:
lda.components_

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([[2.95384921e+01, 2.54900207e-01, 2.53071076e-01, ...,
        1.24035947e+02, 1.05234833e+02, 2.55532560e-01],
       [7.16848028e+01, 2.55663090e-01, 2.53181583e-01, ...,
        2.55748711e-01, 2.56356408e-01, 2.55225909e-01],
       [2.58465999e-01, 2.56439965e-01, 2.53627637e-01, ...,
        4.54722818e-01, 2.54121665e-01, 2.69235590e+02],
       [5.18239060e-01, 2.67232997e+02, 4.81240120e+02, ...,
        2.53581739e-01, 2.54689196e-01, 2.53651467e-01]])

In [ ]:
def display_topics(model, feature_names, num_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic # {topic_idx}")

        topic_word_indexies = topic.argsort()[::-1]
        top_indexies = topic_word_indexies[:num_top_words]
        print(top_indexies)

        feature_concat = " ".join([feature_names[i] for i in top_indexies])
        print(feature_concat)

display_topics(lda, feature_names, 20)

Topic # 0
[ 45 409 106 122 844 918 755 628 109 898 549 620  73 248 856 627 161 562
 989 575]
같아 사람 그거 그럼 친구 하면 정말 요즘 그냥 하는 어떤 완전 결혼 때문 코로나 요새 날씨 언제 회사 없어
Topic # 1
[286 923 678 498 708 466  54 896 774  48 892 759 529 922 108 558 297 762
 242 777]
맞아 하하 이제 아니 있어 시간 거기 하나 좋아하는 같은 하고 제일 않아 하지 그게 어제 머리 조금 들어 좋아해
Topic # 2
[630 610 898 968 116 286 601 905 567 545 911 780  90 639 950 411  45  23
 502 361]
우리 오늘 하는 했어 그런 맞아 영화 하는데 엄마 어디 하더라 좋은 공부 운동 해서 사랑 같아 가족 아니야 별로
Topic # 3
[814 144 430 807 628 665  69 561 708 550 588 753 963 772 390 363 951 325
 950 484]
진짜 나도 생각 지금 요즘 이번 게임 언니 있어 어떻게 여행 정도 했는데 좋아 봤어 보고 해야 무슨 해서 싶다


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## 시각화

In [ ]:
import pyLDAvis.lda_model

pyLDAvis.enable_notebook()
vis = pyLDAvis.lda_model.prepare(lda, feat_vec, count_vectorizer)
pyLDAvis.display(vis)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## 각 문서별 높은 확률의 토픽

In [ ]:
sent_topic = lda.transform(feat_vec)
sent_topic.shape

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(92902, 4)

In [ ]:
sent_topic_per_list = list()

for i in range(sent_topic.shape[0]):
    sent_topic_list = sent_topic[i].argmax()
    topic_per = sent_topic[i].max()
    sent_topic_per_list.append([i, sent_topic_list, topic_per])

topic_per_df = pd.DataFrame(sent_topic_per_list, columns=["index", "topic_index", "probability"])
topic_per_df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,index,topic_index,probability
0,0,2,0.624986
1,1,3,0.849114
2,2,3,0.874412
3,3,1,0.624881
4,4,2,0.561045
...,...,...,...
92897,92897,0,0.562250
92898,92898,2,0.649989
92899,92899,1,0.416677
92900,92900,2,0.624973


In [ ]:
doc_topic_df = topic_per_df.join(df)
doc_topic_df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,index,topic_index,probability,Q,A,label
0,0,2,0.624986,12시 땡!,하루가 또 가네요.,0
1,1,3,0.849114,1지망 학교 떨어졌어,위로해 드립니다.,0
2,2,3,0.874412,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3,1,0.624881,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,4,2,0.561045,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...,...,...,...
92897,92897,0,0.562250,응 탈모는 아니래 ㅎ 숱이 좀 없는 거라고 걱정 말래,내가 너 탈모 아니라고 했잖아,0
92898,92898,2,0.649989,걱정 한시름 덜었다 미용사 언니 덕분에...,NaN,0
92899,92899,1,0.416677,그냥 매직하고 조금 잘랐어,예쁘겠다 사진 보내줘,0
92900,92900,2,0.624973,"모가디슈라는 영화를 봤는데, 재밌더라",맞아. 나도 그거 봤어. 그거 평점이 꽤 높더라,0


In [ ]:
doc_topic_df["topic_index"].value_counts()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


topic_index
0    29161
3    22396
1    21086
2    20259
Name: count, dtype: int64

In [ ]:
for topic in range(len(doc_topic_df["topic_index"].unique())):
    print(f"Topic No: {topic}")

    top_per_topics = doc_topic_df[doc_topic_df["topic_index"]==topic].sort_values(by="probability", ascending=False)

    print(top_per_topics["Q"].iloc[0])
    print(top_per_topics["Q"].iloc[1])
    print(top_per_topics["Q"].iloc[2])

Topic No: 0
2기 나온 거!
원래는 얼마정도 받았지?
너는 연인의 외모를 볼때 어디가 가장 먼저보여?
Topic No: 1
너도 자세 교정이 시급해
와 기숙사면 마스크 벗고 있을텐데 그 사람은 도대체 왜 그랬대…
난 맨날 호텔만 갔었는데 이번에는 에어비앤비 가볼까?
Topic No: 2
재밌어 나중에 시간나면 봐봐
다들 영화나 웹툰 보고나서 별점 달아?
아 진짜? 제주도에 그런 데가 있어?
Topic No: 3
가시에 닿으면 아플 거 같아 하하
썸녀가 오디션 붙었대
웅웅 이제 쓰는 게 편해졌어


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Save

In [ ]:
doc_topic_df.to_csv("/content/drive/MyDrive/메타버스 아카데미/본과정/Python/lda_data.csv")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
import joblib

joblib.dump(lda, '/content/drive/MyDrive/메타버스 아카데미/본과정/Python/lda_model.jl')

# then reload it with
lda_model = joblib.load('/content/drive/MyDrive/메타버스 아카데미/본과정/Python/lda_model.jl')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
